In [1]:
#!pip install qpsolvers
#!pip install cvxopt

     |████████████████████████████████| 269 kB 619 kB/s eta 0:00:01
  Using cached Cython-0.29.22-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
  Created wheel for qpsolvers: filename=qpsolvers-1.5-py3-none-any.whl size=18784 sha256=573ab61b01bf5d4cf3fcc394637ea2e18f27606bb498dfc2400ff4f494b65272
  Stored in directory: /home/bastien/.cache/pip/wheels/f1/ec/63/6cf0699e79ab55a076e8102b075cb9967eda37dc18ec300c6b
  Created wheel for quadprog: filename=quadprog-0.1.8-cp38-cp38-linux_x86_64.whl size=540821 sha256=388d5d1bf306665da24882799edc67b8efef0e03094bc1de01cbc6b1d06eec5a
  Stored in directory: /home/bastien/.cache/pip/wheels/b7/9a/ae/8b1455b942197ab2631f74f0e47dafa1cc1c3aeea28d8953c4
Successfully built qpsolvers quadprog
     |████████████████████████████████| 11.6 MB 9.3 MB/s eta 0:00:01


In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg
from tqdm import tqdm
import numba 
from numba import njit, vectorize, jit
import time 
import scipy

import qpsolvers
from qpsolvers import solve_qp
from qpsolvers import dense_solvers, sparse_solvers

from scipy import optimize

import cvxopt
import cvxopt.solvers

import Estimators
from Estimators import sigmoid, Kernel_cross_val_split

In [2]:
#Pour récupérer toutes les données
X_train0 = pd.read_csv('data/Xtr0_mat100.csv',delimiter= ' ', header= None).values
Y_train0 = pd.read_csv('data/Ytr0.csv',delimiter= ',')['Bound'].to_numpy()
X_test0 = pd.read_csv('data/Xte0_mat100.csv',delimiter= ' ', header= None).values

X_train1 = pd.read_csv('data/Xtr1_mat100.csv',delimiter= ' ', header= None).values
Y_train1 = pd.read_csv('data/Ytr1.csv',delimiter= ',')['Bound'].to_numpy()
X_test1 = pd.read_csv('data/Xte1_mat100.csv',delimiter= ' ', header= None).values

X_train2 = pd.read_csv('data/Xtr2_mat100.csv',delimiter= ' ', header= None).values
Y_train2 = pd.read_csv('data/Ytr2.csv',delimiter= ',')['Bound'].to_numpy()
X_test2 = pd.read_csv('data/Xte2_mat100.csv',delimiter= ' ', header= None).values

X_train = [X_train0,X_train1,X_train2]
X_test = [X_test0,X_test1,X_test2]
Y_train = [Y_train0,Y_train1,Y_train2]

In [3]:
@njit
def GaussianKernel(x, y, sig2 = 1): 
    return np.exp(-numpy.linalg.norm(x-y)**2/(2*sig2))
@njit
def Linear_kernel(x1, x2):
    return np.dot(x1, x2)
@njit
def Polynomial_kernel(x, y, p=5):
    return (1 + np.dot(x, y)) ** p
@njit
def Laplace_kernel(x, y, gamma=1):
    return 0.5 * np.exp(-gamma * numpy.linalg.norm(x-y))

In [4]:
@njit
def to_Kernel_train(X, Kernel, sig2 = 1): 
    length = X.shape[0]
    mat_K = np.zeros((length,length))
    for i in range(length):
        x_i = X[i,:]
        for j in range(i,length): 
            x_j = X[j,:]
            value = Kernel(x_i,x_j,sig2)
            mat_K[i,j] = value
            mat_K[j,i] = value 
    return mat_K

@njit 
def to_Kernel_test(Xtrain,Xtest,Kernel,sig2=1):
    length_train = Xtrain.shape[0]
    length_test = Xtest.shape[0]
    bimat_K = np.zeros((length_train,length_test))
    for i in range(length_train):
        x_i = Xtrain[i,:]
        for j in range(length_test): 
            x_j = Xtest[j,:]
            value = Kernel(x_i,x_j,sig2)
            bimat_K[i,j] = value
    return bimat_K

In [5]:
%time Kernel_train = to_Kernel_train(X_train0,GaussianKernel)

Wall time: 6.11 s


In [6]:
def cross_val_split(Xtrain, ytrain, cv):
    idx = np.arange(Xtrain.shape[0])
    np.random.shuffle(idx) # we shuffle the indices to get random samples
    sample_size = Xtrain.shape[0]//cv
    Xtrainsplit = []# a list that wil contain each X_train vector. Each element will be smaller than 
                    #X_train. If cv = 3 for example, the size (on the x axis) will be 2/3 the original size 
    ytrainsplit = []
    Xvalsplit = []
    yvalsplit = []
    for i in range(cv-1): 
        #we add the new indices. Here, takes the original vector and returns the vector without the 
        # indices passes in argument 
        Xtrainsplit.append(np.delete(Xtrain,idx[i*sample_size:(i+1)*sample_size],axis = 0))
        ytrainsplit.append(np.delete(ytrain,idx[i*sample_size:(i+1)*sample_size],axis = 0))
        
        # we add the rest 
        # note that here, we keep the same labels for X ( we do not shuffle independantly X and y)
        Xvalsplit.append( Xtrain[idx[i*sample_size:(i+1)*sample_size],:])
        yvalsplit.append(ytrain[idx[i*sample_size:(i+1)*sample_size]])
    # we add the last round. It is different since we can't take float proportion of an array, 
    # we have to take an integer. So, here we just add what remains. 
    Xtrainsplit.append(np.delete(Xtrain,idx[(cv-1)*sample_size:],axis = 0))
    ytrainsplit.append(np.delete(ytrain,idx[(cv-1)*sample_size:],axis = 0))
    Xvalsplit.append( Xtrain[idx[(cv-1)*sample_size:],:])
    yvalsplit.append(ytrain[idx[(cv-1)*sample_size:]])
    return Xtrainsplit,Xvalsplit,ytrainsplit,yvalsplit

In [7]:
def download_results(Kernel , cs,name_dossier):
    Y_predicted = []
    for i in range(3) : 
        model = SVM_1(Kernel, C = cs[i])
        model.fit(X_train[i], Y_train[i])
        Y_predicted.append(model.predict(X_test[i]))
        print('Model {} Predicted'.format(i))

    d = { 'Id' : np.arange(3000), 'Bound' : np.concatenate(Y_predicted)}
    out = pd.DataFrame(data=d)
    out.to_csv('predictions_KM'+name_dossier+'.csv', index=False)

In [46]:
for i in range(3): 
    print('cest {}'.format(i))

cest 0
cest 1
cest 2


J'ai fais 2 SVM avec deux technique différentes, et ils donnent des résultats différent, SVM_1 j'ai remplacé alpha*Y pas beta et minimisé par rapport à beta, et SVM_2 c'est le normal comme la formule du cours. Je pense que SVM_1 a des résultats plus logiques, faut prendre C=5 environ.

Au début j'ai voulu faire la méthode du cours mais rien marchait, je me suis donc inspiré d'un truc sur github où ils utilisaient le changement de variable avec beta donc j'ai appliqué ça.

In [87]:
#print(Kernel)

x_train = X_train0[:1500]
y_train = Y_train0[:1500]
x_val = X_train0[1500:]
y_val = Y_train0[1500:]

In [ ]:
class SVM(estimator):
    def __init__(self, Kernel, C = 1):
        self.kernel = Kernel
        self.C = C
        self.alpha = None
        self.b = 0
        
    def fit(self, K, Y):
        #Je fais le changement de variabe: b= alpha * diag(Y)
        #donc je résoud: min_b 1/2 * b.T * diag(Y) * K * diag(Y) * b - b.T * 1 s.t. 0<= b <= C
        n = len(Y)

        #take Y as -1 and 1
        label = 2 * Y - 1
       
        #Create the matrix for the solver
        P = cvxopt.matrix(np.outer(label, label) * K ) 
        q = cvxopt.matrix(-np.ones(n))
        A = cvxopt.matrix(label, (1,n), 'd')
        b = cvxopt.matrix(0.0)
                
        '''Je réécris l'inégalité : 0 <= b <= C avec C = 1/(2*lambda*n)
        comme: G*b <= h avec G = stack(ones(1),-ones(1)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        '''
        # b <= C
        G1 = np.eye(n)
        h1 = np.ones(n) * self.C
        
        # -b <= 0
        G2 = -np.eye(n)
        h2 = np.zeros(n)
        
        G = cvxopt.matrix(np.vstack((G2, G1)))
        h = cvxopt.matrix(np.hstack((h2, h1)))

        #min_b 1/2 * b.T * diag(Y) * K * diag(Y) * b - b.T * 1 s.t. 0<= b <= C
        solver = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        self.beta = np.ravel(solver['x'])
        
        #Je retire les vecteurs avec un beta trop petit
        eps = 1e-5
        supportIndices = np.abs(self.beta) < eps
        ind = np.arange(n)[supportIndices]

        self.alpha = self.beta * label
        self.alpha[supportIndices] = np.zeros(n)[supportIndices]  #On met les alpha à 0 quand ils sont plus petit que eps
        
        #Calcul du Bias
        for i in range(n):
            self.b = label[i]
            self.b -= np.sum( self.alpha * K[i, :])
        self.b /= n

    def set_parameter(self,C): 
        self.C = C

In [103]:
Kernel_train = to_Kernel_train(x_train, GaussianKernel)
Kernel_test = to_Kernel_test(x_train, x_val , GaussianKernel)

In [109]:
model = SVM_1(Kernel = GaussianKernel, C=0.004)
model.fit(Kernel_train, y_train)

In [110]:
prediction_SVM = model.predict(Kernel_test)

In [111]:
print(prediction_SVM[0:100])
print(sum(prediction_SVM == y_val)/500)

[1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0
 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 0 1]
0.576


In [107]:
def grid_search_cv(model, Kernel_train, Y_train, parameters): 
    '''
    grid search cv. 
    put the model you want, for example Estimators.KRR(), the Kernel_train you want to train on, the labels Y_train
    and the list of parameters (parameters) you want to try. 
    
    Will print the best parameter with the score associated. 
    '''
    scores = list()
    for parameter in parameters : 
        model.set_parameter(parameter)
        scores.append(model.cross_val(Kernel_train,Y_train, 5))
        print('We have tested  : ', np.round(parameter, 6))
    arg_max = np.argmax(np.array(scores))
    print('###########BEST_PARAM = ',np.round(parameters[arg_max], 6), 'with score :', scores[arg_max])
    return parameters[arg_max]

In [95]:
Kernel_cv = to_Kernel_train(X_train[0], GaussianKernel)

In [100]:
parameters = np.linspace(0.1,0.001,8)
grid_search_cv(model, Kernel_cv, Y_train[0], parameters)

5it [00:19,  3.87s/it]


Score :  0.554
We have tested  :  0.1


5it [00:19,  3.91s/it]


Score :  0.5529999999999999
We have tested  :  0.085857


5it [00:19,  3.82s/it]


...

5it [00:19,  3.91s/it]


Score :  0.569
We have tested  :  0.015143


5it [00:18,  3.78s/it]

Score :  0.5185
We have tested  :  0.001
###########BEST_PARAM =  0.015143 with score : 0.569


0.015142857142857152

In [60]:
def grid_search_C(X, Y, C_list, Kernel, cv = 4):
    acc = []
    for c in C_list:
        print('____ For C = ', c, '____')
        model = SVM_1(Kernel, C=c)
        acc.append(model.cross_val(X, Y, cv))
    best_param = C_list[np.argmax(acc)]
    print('The best value for C is', best_param, 'we get', np.max(acc))
    return best_param


In [58]:

cs = [0.003, 20, 5]
download_results(GaussianKernel, cs,'first')

We keep  1939 support vectors out of 2000 vectors
Model 0 Predicted
We keep  1866 support vectors out of 2000 vectors
Model 1 Predicted
We keep  1696 support vectors out of 2000 vectors
Model 2 Predicted


In [61]:
%time best_c = grid_search_C(X_train1, Y_train1, [1], Kernel = Polynomial_kernel)

0it [00:00, ?it/s]

____ For C =  1 ____
We keep  1477 support vectors out of 1500 vectors


1it [01:57, 117.91s/it]

We keep  1485 support vectors out of 1500 vectors


2it [04:04, 122.80s/it]

We keep  1469 support vectors out of 1500 vectors


3it [06:26, 131.54s/it]

We keep  1479 support vectors out of 1500 vectors


4it [08:35, 128.84s/it]

Pourcentage of errors :  0.4285
The best value for C is 1 we get 0.4285
CPU times: user 9min 5s, sys: 32.3 s, total: 9min 38s
Wall time: 8min 35s


In [164]:
model = SVM_1(kernel = GaussianKernel, C=5)

model.fit(x_train, y_train)

results = model.predict(x_val)

print( results[:100], y_val[:100])
n = results == y_val
print('accuracy:', sum(n)/len(results))
#print(model.alpha, len(model.alpha_), model.b)

We keep  1304 support vectors.
[1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0
 1 1 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 0
 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 0 1 1 0] [0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0
 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 0 1
 1 1 0 1 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1]
accuracy: 0.672


### Autre SVM (pas besoin car SVM_1 fonctionne très bien):

In [108]:
#avec cvxopt pour résoudre min_b 1/2 * b.T * diag(Y) * K * diag(Y) * b - b.T * 1 s.t. 0<= b <= C  
#(en gros b= alpha * diag(Y))
class estimator(): 
    def __init__(self , Kernel): 
        self.Kernel = Kernel
        self.Kernel_train = None 
        self.alpha = None
        self.b = 0
        
    def predict_proba(self,Kernel_test): 
        if (self.alpha == None).any()==True  : 
            print("Il faut d'abord fitter les données")
        else : 
            return  sigmoid(self.alpha@Kernel_test + self.b)
    
    def predict(self,K_test): 
        if (self.alpha == None).any()==True : 
            print("Il faut d'abord fitter les données")
        else : 
            prob = self.predict_proba(K_test)
            return (prob>0)*1
        
    def cross_val(self, K_train,ytrain,cv): 
        mistake = 0
        Kerneltrainsplit,Kernelvalsplit,ytrainsplit,yvalsplit = Kernel_cross_val_split(K_train,ytrain,cv)
        for Ktrain,Kval,ytrain,yval in tqdm(zip(Kerneltrainsplit,Kernelvalsplit, ytrainsplit, yvalsplit)):
            self.fit(Ktrain,ytrain)
            pred = self.predict(Kval)
            mistake+=np.sum(np.abs(pred-yval))
        print('Score : ',1- mistake/K_train.shape[0])
        return 1- mistake/K_train.shape[0]





class SVM_1(estimator):
    def __init__(self, Kernel, C = 1):
        self.kernel = Kernel
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_Y = None
        
    def fit(self, K, Y):
        n = len(Y)
        #calculate the kernel
        #K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : self.kernel(x2, x1), 1, X), 1, X)
    
        lbd = 1
        #C = 1 / (2 * n * lbd)  #ça dépend si on veut gérer C ou lambda

        #take Y as -1 and 1
        label = 2 * Y - 1
        
        
        P = cvxopt.matrix(np.outer(label, label) * K ) 
        q = cvxopt.matrix(-np.ones(n))
        A = cvxopt.matrix(label, (1,n), 'd')
        b = cvxopt.matrix(0.0)
                
        '''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
        comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        ça revient au même et je crois que le solver devrait fonctionner avec ça, mais j'y arrive pas encore
        '''
        # b <= C
        G1 = np.eye(n)
        h1 = np.ones(n) * self.C
        
        # -b <= 0
        G2 = -np.eye(n)
        h2 = np.zeros(n)
        
        G = cvxopt.matrix(np.vstack((G2, G1)))
        h = cvxopt.matrix(np.hstack((h2, h1)))

        #min_b 1/2 * b.T * diag(Y) * K * diag(Y) * b - b.T * 1 s.t. 0<= b <= C
        solver = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        self.beta = np.ravel(solver['x']) # le alpha ou on garde toutes les coordonnées 
                                                # en comparaison avec le self.alpha ou en garde que quelques-uns 
        
        #Je retire les vecteurs avec un alpha trop petit
        eps = 1e-5
        supportIndices = np.abs(self.beta) < eps
        ind = np.arange(n)[supportIndices]
        
        #self.support_vectors = X[supportIndices]
        #self.support_Y = label[supportIndices]
        #self.alpha = self.beta[supportIndices]  
        #alpha : all_alpha sans les alpha < eps        
        #print('We keep ', len(self.alpha), 'support vectors out of',len(self.all_alpha),'vectors')
        
        #Pour prédire avec classe estimator:
        #self.X_train = X
        self.alpha = self.beta*label
        self.alpha[supportIndices] = np.zeros(n)[supportIndices]
        
        #Bias
        self.b = 0
        for i in range(n):
            self.b = label[i]
            self.b -= np.sum( self.alpha * K[i, :])
        self.b /= n
    
    def predict2(self, X):
        
        y_predict = np.zeros(len(X))
        
        for i in range(len(X)):
            for alpha, sv, label in zip(self.alpha, self.support_vectors):
                y_predict[i] += alpha * self.kernel(sv, X[i]) 
        
        return ((y_predict + self.b) > 0)*1
        #return y_predict + self.b
        
    def set_parameter(self,C): 
        self.C = C 
    '''
    def cross_val(self, Xtrain, ytrain, cv): 
        mistake = 0
        Xtrainsplit, Xvalsplit, ytrainsplit, yvalsplit = cross_val_split(Xtrain, ytrain, cv)
        for xtrain, xval, ytrain, yval in tqdm(zip(Xtrainsplit, Xvalsplit, ytrainsplit, yvalsplit)):
            self.fit(xtrain, ytrain)
            pred = self.predict(xval)
            print('Accuracy :', 1 - np.sum(np.abs(pred - yval)) / pred.shape[0])
            mistake += np.sum(np.abs(pred - yval))
        print('Average accuracy: ', 1 - mistake / Xtrain.shape[0])
        return 1 - mistake / Xtrain.shape[0]
    '''
    
#model = SVM_1(Kernel = GaussianKernel, C=5)


In [59]:
#avec cvxopt pour résoudre min_a 1/2 * alpha.T * K * alpha - alpha.T * Y s.t. 0<=y*alpha<=C
class SVM_2:
    def __init__(self, kernel = GaussianKernel, C = 1):
        self.kernel = kernel
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_Y = None
        
        
    def fit(self, X, Y):
        n = len(Y)        
        #K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : self.kernel(x2, x1), 1, X), 1, X)
                
        K = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                K[i,j] = self.kernel(X[i], X[j])

        #lbd = 1
        #C = 1 / (2 * n * lbd)  #ça dépend si on veut gérer C ou lambda

        label = 2 * Y - 1
        
        # P=K et q=-Y
        P = cvxopt.matrix(K) 
        q = cvxopt.matrix(-label, tc='d')
                
        '''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
        comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        '''
        
        # Condition 0 <= alpha_i * Y_i <= C
        G1 = np.diag(-label)
        G2 = np.diag(label)
        G = cvxopt.matrix(np.vstack((G1, G2)), tc='d')
        
        h1 = np.zeros((n, 1), dtype='float64')
        h2 = self.C * np.ones((n, 1), dtype='float64')
        h = cvxopt.matrix(np.vstack((h1, h2)))

        # solves min_a 1/2 a^T * P * a + q^T * a s.t. G*a <= h
        solver = cvxopt.solvers.qp(P, q, G, h)
        
        self.alpha = np.ravel(solver['x'])
        
        #Je retire les vecteurs avec un alpha trop petit
        eps = 1e-5
        supportIndices = np.abs(self.alpha) > eps
        ind = np.arange(n)[supportIndices]
        
        self.support_vectors = X[supportIndices]
        self.support_Y = label[supportIndices]
        self.alpha_ = self.alpha[supportIndices]  #alpha_ : alpha sans les alpha < eps
        
        #Bias
        self.b = 0
        for i in range(len(self.alpha_)):
            self.b = self.support_Y[i]
            self.b -= np.sum( self.alpha_ * K[ind[i], supportIndices])
        self.b /= len(self.alpha_)
    
    def predict(self, X):
        
        y_predict = np.zeros(len(X))
        
        for i in range(len(X)):
            for j in range(len(self.alpha_)):
                y_predict[i] += self.alpha_[j] * self.kernel(self.support_vectors[j], X[i])
        
        return ((y_predict + self.b) > 0)*1
        #return y_predict + self.b
        

In [74]:
model = SVM_2(kernel = GaussianKernel, C=25)

model.fit(x_train, y_train)

results = model.predict(x_val)

print( results[:100], y_val[:100])
n = results == y_val
print(sum(n))
print(model.alpha, len(model.alpha_), model.b)

EN dessous il y a un autre avec la même méthode que 'SVM_1' mais avec un autre optimiseur.

In [11]:
#Avec qp_solver pour résoudre min_b 1/2 * b.T * diag(Y) * K * diag(Y) * b - b.T * 1 s.t. 0<= b <= C  
#(en gros b= alpha * diag(Y))

#Il marche et donne le même résultat que cvxopt

class SVM_autre:
    def __init__(self, kernel=GaussianKernel, C=1):
        self.kernel = kernel
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_Y = None
        
        
    def fit(self, X, Y):
        n = len(Y)        
        K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : self.kernel(x2, x1), 1, X), 1, X)
        
        #C = 1 / (2 * n * lbd)  #ça dépend si on veut gérer C ou lambda

        label = 2 * Y - 1
        
        P = np.outer(label, label) * K
        q = - np.ones(n)
        
        '''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
        comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        ça revient au même et je crois que le solver devrait fonctionner avec ça, mais j'y arrive pas encore
        '''
        G = np.vstack((np.eye(n), -np.eye(n)))

        h = np.ones(2*n)
        h[:n] = h[:n] * self.C
        h[n:] = h[n:] * 0

        A = label
        b = np.array([0.])

        self.alpha = solve_qp(P, q.astype('double'), G.astype('double'), h, A, b, solver = 'quadprog')
        
        #Pour le moment je garde tout X mais faudrait retirer les X dont le alpha est trop bas
        eps = 1e-15
        supportIndices = self.alpha > eps
        ind = np.arange(n)[supportIndices]
        
        self.support_vectors = X[supportIndices]
        self.support_Y = label[supportIndices]
        self.alpha_ = self.alpha[supportIndices]
        
        #Bias
        self.b = 0
        for i in range(len(self.alpha_)):
            self.b = self.support_Y[i]
            self.b -= np.sum( self.alpha_ * self.support_Y * K[ind[i], supportIndices])
        self.b /= len(self.alpha_)
    
    def predict(self, X):

        y_predict = np.zeros(len(X))

        for i in range(len(X)):
            for alpha, sv, label in zip(self.alpha_, self.support_vectors, self.support_Y):
                y_predict[i] += alpha * label * self.kernel(sv, X[i]) 

        return ((y_predict + self.b) > 0)*1
    
    def predict_bis(self, X):
        
        def predict_one(x):
            pred = np.apply_along_axis(lambda s: self.kernel(s, x), 1, self.support_vectors)
            pred = pred * self.support_Y * self.alpha
            return np.sum(pred)
        
        preds = np.apply_along_axis(predict_one, 1, X)
        return 1 * (preds > 0)
        
        

In [38]:

svm = SVM_autre(kernel = GaussianKernel, C=0.01)
svm.fit(x_train, y_train)


In [39]:
results = svm.predict(x_val)
print(results[:100], y_val[:100])
print(svm.alpha_)

[1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 1
 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1
 1 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1] [0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1
 0 1 0 0 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1
 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0]
[0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01     

Aide svm github : https://github.com/zongmianli/mva-kernel-methods/blob/kernel-challenge/svm.py

In [132]:
#Tests sur les fct de prédiction

def predict_one(x):
    pred = np.apply_along_axis(lambda s: GaussianKernel(s, x), 1, X_bis)
    pred = pred * Y_bis * model.alpha
    return np.sum(pred)

def f_from_alpha(alpha, Kernel, X):
    return  lambda x : np.sum([alpha[i]*Kernel(X[i,:],x) for i in range(X.shape[0])])
f_alpha = f_from_alpha(model.alpha, GaussianKernel, x_train)

res = [f_alpha(x_val[i]) for i in range(len(y_val))]

preds = np.apply_along_axis(predict_one, 1, x_val)
#preds = 1 * (preds > 0)
print(preds, y_val, res)

[ 0.02414692  0.00498681  0.01534296  0.03327065  0.02499289  0.01782278
  0.00641423  0.01737927  0.00571243  0.03785679  0.02937389 -0.0020743
  0.01801311  0.01202195  0.02616614  0.02493651  0.02069579  0.00564455
 -0.00066408  0.02225113  0.00237909  0.03542867  0.00787691  0.02247312
  0.02751154  0.00633758  0.01760011  0.0165434   0.0267509   0.03206684
  0.01438477  0.03242849  0.01270201  0.01455076  0.00451208  0.01183673
  0.01354231  0.00326173  0.04398204  0.02790272 -0.00445432  0.04707981
  0.00802263  0.02043321  0.00878177  0.01557869  0.03094886  0.03406918
  0.02125684  0.02878366  0.00324311  0.01825782  0.02390436  0.00269406
  0.02602237  0.01234005  0.00085998  0.0158754   0.02489576  0.02026007
  0.00903557  0.0151773   0.03625054  0.01268538  0.00022868  0.00617082
  0.0134089   0.01244844 -0.00563212  0.00012831  0.02412231  0.02177587
  0.01969552  0.03197964  0.01770224  0.01614737  0.01016502  0.03356141
  0.00210028  0.02871426  0.00812573 -0.00372503  0.